In [ ]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.datasets import cifar10
from keras.callbacks import LearningRateScheduler
from keras.models import model_from_json
from keras.models import load_model
import numpy as np
import keras
import math
import time
import matplotlib.pyplot as plt
import sys
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

from ourwrnet import create_wide_residual_network
from cifar10utils import getCIFAR10, getCIFAR10InputShape


'''
Function that loads from a file the teacher
'''
def getTeacher(file_name):
    # Model reconstruction from JSON file
    with open(file_name + '.json', 'r') as f:
        model = model_from_json(f.read())

    # Load weights into the new model
    model.load_weights(file_name + '.h5')
    
    print('Teacher loaded from' + file_name + '.h5')
    return model
    
'''
Function that loads from a file the teacher and test it on the CIRAF10 dataset
'''
def testTeacher(file_name):
    x_train,y_train,x_test,y_test = getCIFAR10()
    model = getTeacher(file_name)
    opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=opt_rms, metrics=['accuracy'])
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Teacher test loss:', score[0])
    print('Teacher test accuracy:', score[1])
    
'''
Function that returns a simple student done by 2 convolutions, a maxpool and a final two fully connected layers
'''
def getStudent(input_shape):
    num_classes = 10
    
    model_train,model_test,m1,m2,m3 = create_wide_residual_network(input_shape, nb_classes=10, N=2,k=1)
    
    print('Simple student loaded')
    return model_train, model_test

'''
Function that returns a simple generator
'''
def getGenerator():

    noise_shape = (100,)

    model = Sequential()

    img_shape = getCIFAR10InputShape()

    model.add(Dense(128*8**2, input_shape=noise_shape))
    model.add(Reshape((8, 8, 128)))
    model.add(BatchNormalization())

    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2)) 

    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=(3,3), strides=1, padding="same"))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(3, kernel_size=(3,3), strides=1, padding="same"))
    model.add(BatchNormalization())   

    #model.summary()
    print('Generator loaded')
    
    return model

def getGAN(teacher,student,generator):
    z = Input(shape=(100,))
    img = generator(z)
    student.trainable = False # it works if it is not true
    teacher.trainable = False
    
    out_t = teacher(img)
    out_s = student(img)
    
    joinedOutput = Concatenate()([out_t,out_s])
    
    gan = Model(z,joinedOutput)
    
    return gan

def gan_loss(y_true, y_pred):
    t_out = y_pred[:,0:10]
    s_out = y_pred[:,10:21]
    
    loss = keras.losses.kullback_leibler_divergence(t_out,s_out)
    min_loss = (-(loss))
    
    return min_loss
        
def main():
    
    x_train,y_train,x_test,y_test = getCIFAR10()
    input_shape = getCIFAR10InputShape()
    
    teacher = getTeacher('./pretrained_models/wrn_16_2')
    teacher.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
    
    optim_stud = Adam(lr=2e-3, clipnorm=5.0)
    optim_gen = Adam(lr=1e-3, clipnorm=5.0)
    
    student_train, student_test = getStudent(input_shape)
    
    student_test.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
    
    student_train.compile(loss=keras.losses.kullback_leibler_divergence, optimizer=optim_stud)
    
    generator = getGenerator()
    
    gan = getGAN(teacher,student_train,generator)
    gan.summary()
    
    gan.compile(loss=gan_loss, optimizer=optim_gen)
    
    n_batches = 1000
    batch_size = 128
    log_freq = 10
    ns = 5
    
    print('TEACHER SUMMARY')
    teacher.summary()
    print('TEACHER L1 SUMMARY')
    t_layer1.summary()
    print('TEACHER L2 SUMMARY')
    t_layer2.summary()
    print('TEACHER L3 SUMMARY')
    t_layer3.summary()
    
    print('STUDENT SUMMARY')
    student.summary()
    
    noise = np.random.normal(0, 1, (batch_size, 100))
    gen_imgs = generator.predict(noise)
    t_final_out = teacher.predict(gen_imgs)
    print('Teacher output shape: ' + str(t_final_out))
    t_l1_out = t_layer1.predict(gen_imgs)
    print('Teacher LAYER1 output shape: ' + str(t_l1_out))
    t_l2_out = t_layer2.predict(gen_imgs)
    print('Teacher LAYER2 output shape: ' + str(t_l2_out))
    t_l3_out = t_layer3.predict(gen_imgs)
    print('Teacher LAYER3 output shape: ' + str(t_l3_out))
    
    student_all_outputs = student_train.predict(gen_imgs)
    for i in range (len(student_all_outputs)):
        print('Student output number ' + str(i) + ' shape: ' + student_all_outputs[i].shape)
 
    
    '''
    for i in range(n_batches):
        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)
        t_predictions = teacher.predict(gen_imgs)
        
        fake_lbl = K.zeros((batch_size,20))
        g_loss = gan.train_on_batch(noise,fake_lbl)
        s_loss = 0
        for j in range(ns):
            s_loss += student_train.train_on_batch(gen_imgs,t_predictions)
        
        print('batch ' + str(i) + '/' + str(n_batches) + ' G loss: ' + str(g_loss) + ' S loss: ' + str(s_loss/ns))
        
        if (i % log_freq) == 0:
            score = student_test.evaluate(x_test, y_test, verbose=0)
            print('Student test loss: '  + str(score))
        
        
    score = student_test.evaluate(x_test, y_test, verbose=0)
    print('Student test loss: '  + str(score))
    '''

main()


170500096/170498071 [==============================] - 34s 0us/step
CIFAR10 loaded


In [ ]:
import keras
from keras import backend as K

#a = K.zeros((2,4))
a = K.random_uniform((2,4))

loss = keras.losses.kullback_leibler_divergence(a,a)

with tf.Session() as sess:
                init = tf.global_variables_initializer()
                sess.run(init)
                print('tensor: ' + str(a.eval()))
                print('loss: ' + str(loss.eval()))